<a href="https://colab.research.google.com/github/kmk4444/LLM/blob/main/Basit_LLM_Modellerini_Kullanma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Giriş ve Ortam Hazırlığı

## 1.1 Ön Hazırlık ve Gerekli Kütüphanelerin Yüklenmesi

Aşağıda **versiyonları** ile verilen Python kütüphanelerini pip install ile yükleyebiliriz ya da içeriği, dosya bölümünde (ya da lokalimizdeki çalışma dizinimize) requirements.txt dosyası olarak kaydedip  `pip install -r requirements.txt` terminal komutu ile yükleyebiliriz.

- **requirements.txt**
```text
openai==1.3.7
langchain==0.0.345
tiktoken==0.5.2
cohere==4.37
python-dotenv==1.0.0
```
python-dotenv sadece lokalimizde .env ortam değişkenleri (api key vb) kullanmak için gerekiyor. Yani Google Colab üzerinden çalışacağımız zaman python-dotenv kütüphanesine ihtiyacımız yok.

> Google Colab ile ilgili tutorialı [izlemek için tıklayın](https://youtu.be/hzc5mdh7TOI?si=cRXC_sqI1x7GlHVH)

- **terminal / bash komutu**
```bash
pip install -r requirements.txt
```


- Terminal Üzerinden requirements.txt dosyasını oluşturuyoruz (copy paste de yapabiliriz.)

In [ ]:
!touch requirements.txt
!echo 'openai==1.3.7' >> requirements.txt
!echo 'langchain==0.0.345' >> requirements.txt
!echo 'cohere==4.37' >> requirements.txt
!echo 'tiktoken==0.5.2' >> requirements.txt

- requirements.txt dosyası içeriğindeki paketleri runtime'ımıza yüklüyoruz

In [ ]:
!pip install -r requirements.txt -q

## 1.2 Paketlerimizi ve yüklemelerimizi kontrol edelim.

In [ ]:
!pip show langchainXYY

In [ ]:
import openai, langchain

In [ ]:
print(openai.__version__)
print(langchain.__version__)

1.3.7
0.0.345


In [ ]:
%%html
<marquee style='width: 50%; color: Green;'>Tebrikler, buraya kadar hatasız geldiyseniz, oldu bu iş... gerisi kodlama!</marquee>

In [ ]:
!pip freeze

In [ ]:
!pip freeze > requirements.txt

In [ ]:
%history

## 1.3 Open AI API Key'imizi alma ve gizli anahtarlar bölümüne kaydetme

Öncelikle OpenAI'ın web sitesinde kayıt olmamız ve bir API key oluşturmamız gerekiyor. Tabii ki bunun içinde geçerli bir hesabınız olması gerekiyor. Colab notebook'u kullandığınız gmail adresi ile çok hızlı bir şekilde login / signup yapabilirsiniz.

> ::ÇOK ÖNEMLİ:: OpenAI API'ı ile yapacağımız tüm işlemler ücretli olacak. Dolayısıyla emin olmadığınz hareketlerden şimdilik kaçınmanız çok rasyonel olur. Zamanla zaten bu konularda da hakimiyet kazanacaksınız

- OpenAI Giriş Sayfası: [link](https://openai.com/)
- OpenAI Ürünleri Anasayfa: [link](https://openai.com/product)
- OpenAI Ücretlendirmeleri: [link](https://openai.com/pricing)
- OpenAI Tokenizer (Tokenların maliyetini hesaplama): [link](https://platform.openai.com/tokenizer)
- OpenAI Dokümantasyonu: [link](https://platform.openai.com/docs/introduction)

### 1.3.1 Google Colab'de API Anahtarımızı Kaydetme


Yapmamız gereken tek şey ekranın sol tarafındaki panelde, anahtar işaretini tıklamak ve Ad kısmına OPENAI_APIKEY, değer kısmına da API keyimizin kendisini yazmamız; ve aşağıdaki kod ile bu  api anahtarını bir değişken içerisine kaydetmemiz.

Aşağıdaki kodu çalıştırdığımızda bizden erişim izni isteyecek (tabii ki anahtarlar kısmında erişim vermediysek)

```python
from google.colab import userdata

OPENAI_APIKEY = userdata.get('secretName')

```


In [ ]:
from google.colab import userdata
openai_apikey = userdata.get('OPENAI_APIKEY')

In [ ]:
openai_apikey

### 1.3.2 Lokalimizde nasıl yapacağız?

Lokalimizde çalışma dizinimizde .env isimli bir dosya oluşturup içeriğine aşağıdaki şekilde api key'imizi yazamamız lazım

- .env içeriği
```text
OPENAI_APIKEY='sk-APIANAHTARIMIZBURADA OLACAK'
```


- Python ile api anahtarımıza ulşma: Daha sonra, kod editörümüzde ya da jupyter notebook'umuz aşağıdaki kod snippetini çalıştırmamız lazım

```python
from dotenv import load_dotenv, find_dotenv #pip install python-dotenv
import os
load_dotenv(find_dotenv(), override=True)

OPENAI_APIKEY = os.environ.get('OPENAI_APIKEY', 'bulunamaz_default_değer')
# ya da default bir değer olmadan
OPENAI_APIKEY = os.environ.get('OPENAI_APIKEY')
```


###  1.3.3 Colab Magiclerle Nasıl Yapabiliriz

Hücrelerimizde colab magic komutlarını başına '%' işareti getirerek kullanabiliyoruz.

Örnek:
```python
%env OPENAI_APIKEY='asdfasdfasdfasfdasdf'
import os
sample_key = os.environ.get('OPENAI_APIKEY')
```

Tüm magic listesini görmek için sadece hücreye %magic yazın ve çalıştırın

---

# 2 Basit LLM Modelleri ile Çalışma

> Tabii ki eğer ara verip notebook'umuzu tekrar başlattıysak ilgili importlarımızı yapmayı unutmuyoruz.

LangChain, çeşitli türde modellerle çalışabilme yeteneğine sahiptir. Bu bölümde, üç temel model türünü ele alacağız:
1. Basit LLM,
2. Chat (Sohbet )
3. Embedding (Gömme) modelleri

Neden langchain kullanalım ki, zaten bunları mevcut model sağlayıcılar (OpenAI vb.) ile de yapabiliyoruz, hocam iş çıkarma başımıza diyenler için  [ilgili video ve cevap](https://youtu.be/x7RkjTwkZ5Y?si=mRZAU73Li3rB0GvQ)

## 2.1 Basit LLM'ler
Bir text girdisi gönderip, bir text çıktısı almak. Buradaki basit kelimesi hiç birimizi yanıltmasın; çünkü, prompt'unuz (yani isteminiz) ne kadar kuvvetliyse, basit modeliniz bile o derece kuvvetlidir.

Ayrıca, bir diğer önemli konu, artık hem Google Cloud, hem de OpenAI tarafında kendi modellerimizi eğitim "fine tune" edebilmemiz. Bu ne demek oluyor? Çok basit, eğer siz hazır bir modeli alırsanız ve belli prompt ve cevaplarla eğitirseniz, göndereceğiniz bir kaç kelimelik promptlar ile bile çok güçlü yanıtlar alabilirsiniz. O yüzden ne diyoruz:

> Basit Model yoktur, kötü prompt vardır.

İlerleyen derslerimizde zaten, bu konuyu çoook derinlemesine irdeleyeceğiz ✊

Bakmak isteyenler için mevcut openai modellerine [bu bağlantıdan erişilebilir](https://platform.openai.com/docs/models/gpt-3-5)

In [ ]:
from google.colab import userdata
from langchain.llms.openai import OpenAI
from pprint import pprint


OPENAI_APIKEY = userdata.get('OPENAI_APIKEY')

llm = OpenAI(
    api_key=OPENAI_APIKEY,
    model='text-davinci-003',
    temperature=0.7,
    max_tokens=1024,
    max_retries=2
)
print(llm)
### İlgili Colab Magicleri
# %pdoc OpenAI
# %pfile OpenAI
# %pinfo llm


OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 1024}


#### Bu çıktı Bize ne ifade ediyor?


Yukarıdaki kodun çıktısı aşağıdakine bezer birşeyler olacak
```json
Params:
  {
    'model_name': 'text-davinci-003',
    'temperature': 0.7,
    'top_p': 1,
    'frequency_penalty': 0,
    'presence_penalty': 0,
    'n': 1,
    'logit_bias': {},
    'max_tokens': 512
    }
```

Her bir parametreyi açıklayalım:

1. **`api_key`**: Bu, OpenAI API'sine erişmek için kullanılan özel anahtardır. `os.environ.get` fonksiyonu, bu anahtarı ortam değişkenlerinden alır. Eğer bulunamazsa 'NotFound' olarak ayarlanır.

2. **`model`**: Burada `'text-davinci-003'` belirtilmiştir. Bu, OpenAI tarafından sağlanan GPT-3 modelinin bir versiyonunu ifade eder. Bu model, çeşitli dil işleme görevleri için kullanılır ve oldukça genel amaçlıdır. Mevcut openai modellerine [bu bağlantıdan erişilebilir](https://platform.openai.com/docs/models/gpt-3-5)

3. **`temperature`**: Bu değer, modelin ürettiği cevapların özgünlük derecesini belirler. `0.7` değeri, orta derecede özgün yanıtlar üretilmesini sağlar. Düşük bir sıcaklık değeri daha tahmin edilebilir ve tutarlı cevaplar üretirken, yüksek bir değer daha yaratıcı ve beklenmedik yanıtlara yol açar.

4. **`max_tokens`**: Bu, modelin üretebileceği maksimum token (kelime veya kelime parçaları) sayısını belirtir. `512` değeri, modelin bir seferde en fazla 512 token üretebileceği anlamına gelir. Bu, cevap uzunluğunu sınırlar.

5. **`max_retries`**: Bu, bir isteğin başarısız olması durumunda maksimum yeniden deneme sayısını belirtir. `5` olarak ayarlanmış, bu da isteğin başarısız olması durumunda en fazla 5 kez yeniden denenmesi gerektiği anlamına gelir.

6. **`top_p`**: Bu, modelin cevap üretirken dikkate alacağı olası sonraki tokenlerin yüzdesini belirler. `1` olarak ayarlandığında, model, bir sonraki tokeni seçerken tüm olası tokenleri dikkate alır.

7. **`frequency_penalty`** ve **`presence_penalty`**: Bu iki parametre, modelin cevapları üretirken belirli kelimeleri tekrarlamasını engellemeye yardımcı olur. Her ikisi de `0` olarak ayarlanmıştır, bu da cevaplarda herhangi bir kelime tekrarlama veya varlık cezasının uygulanmayacağı anlamına gelir.

8. **`n`**: Bu, modelin bir seferde üreteceği cevap sayısını belirtir. `1` olarak ayarlanmış, bu da her istekte modelin tek bir cevap üreteceği anlamına gelir.

9. **`logit_bias`**: Bu, belirli tokenlerin olasılığını artırmak veya azaltmak için kullanılabilir. Boş bir sözlük `{}` olarak ayarlanmış, bu da herhangi bir önyargı uygulanmayacağı anlamına gelir.

Bu parametreler, OpenAI'nin GPT-3 modelini kullanarak daha kontrollü ve özelleştirilmiş cevaplar üretmek için kullanılır.



### 2.1.2 Basit bir sorgu atalım

Karşılaşabileceğiniz Hatalar ile ilgili Uyarılar:

- API anahatarınız doğru değilse, ya da ulaşılamıyorsa: AuthenticationError: Error code: 401
- Tanımlı ücretsiz kredi kullanıyorsanız ya da çok fazla istekte bulunduysanız: RateLimitError ve bir mesaj alırsınız:  You exceeded your current quota, please check your plan and billing details.

In [ ]:
# LLMizi başlatalım
from google.colab import userdata
from langchain.llms.openai import OpenAI
from pprint import pprint


OPENAI_APIKEY = userdata.get('OPENAI_APIKEY')

llm = OpenAI(
    api_key=OPENAI_APIKEY,
    model='text-davinci-003',
    temperature=0.7,
    max_tokens=2048,
    max_retries=2
)

In [ ]:
soru = """
İklim değişikliği hakkında bilgi verir misin?
"""
cevap = llm(soru)
%pycat cevap

In [ ]:
soru = """
Sen bir akademisyen ve çok iyi bir iklim bilimcisin, iklim değişikliği hakkında bilgi verir misin?
"""
cevap = llm(soru)
%pycat cevap

#### Peki kaç token gitti?

In [ ]:
llm.get_num_tokens(soru)

25

### 2.1.2 Birden fazla sorgu içeriği

In [ ]:
output = llm.generate(
    [
        "... Türkiye'nin başkentidir",
        "Pi sayısı kaçtır?",
        "'Hadi oradan lan?' cümlesinin ingilizce karşılığı nedir?"
      ]
)

In [ ]:
output

LLMResult(generations=[[Generation(text=' Ankara.\n\nAnkara is the capital of Turkey.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text="\n\nPi sayısı 3.1415926535897932384626433832795'dir.", generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Did he leave from there?"', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 53, 'total_tokens': 100}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('63f22ed9-a063-4c02-a2f8-4009f056b07b')), RunInfo(run_id=UUID('733a645c-bfbb-4756-a22b-dc7db9adfc1c')), RunInfo(run_id=UUID('339966c5-a9bc-405a-b0ca-535ef93bcad8'))])

In [ ]:
output.generations[-1][0].text

'\n\n"Did he leave from there?"'

In [ ]:
for i in output.generations:
  print(i[0].text)

 Ankara.

Ankara is the capital of Turkey.


Pi sayısı 3.1415926535897932384626433832795'dir.


"Did he leave from there?"


In [ ]:
output_multiple  = llm.generate(['Langchain youtube videosu için isim önerisi'] * 3)
print(len(output_multiple.generations))

3


In [ ]:
for i in output_multiple.generations:
  print(i[0].text)



Langchain: Dili Değiştir, Dünyayı Dönüştür.


Eylem İçin Blockchain: Bir Kılavuz


"Langchain: Dil Öğrenmeyi Kolaylaştıran Sistem"


## 2.2 Chat Modelleri  GPT-3.5-Turbo and GPT-4

> Burada biraz karmaşık konulara girmeye başlıyoruz, ancak OpenAI sitesinde "Playground" bölümünde yaptığımız çalışma ve video buradaki tüm soru işaretlerinizi giderecek". Mutlaka izleyin: izlemek için [bu linki tıklayın](https://youtube.com).


> **Don't worry! Be Happy, you are with me***

Sohbet modelleri, girdi olarak bir mesaj listesi alır ve çıktı olarak model tarafından üretilmiş bir mesaj döndürür. Sohbet formatı, çok turlu konuşmaları kolaylaştırmak için tasarlanmış olsa da, herhangi bir konuşma olmadan tek turda gerçekleşen görevler için de oldukça yararlıdır.




Bir örnek Chat Completions API çağrısı şu şekilde görünür:

```python
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
```

The system asistanımızın yani modelimizin davranışını vereceği sonuçları belirlediğimiz kısım.
- System rolü, asistanın davranışını belirler.
- User rolü, bizim cevabını aradığımız soru
- Assistant rolü, LLM'in bize gönderdiği cevap

link = https://platform.openai.com/docs/guides/text-generation/chat-completions-api

In [ ]:
from google.colab import userdata
from langchain.schema import (
    AIMessage,  # Asistan mesajı yani cevap
    HumanMessage, # Bizim mesajımız / sorumuz
    SystemMessage, # Sistem mesajı yani  asistanın davranışını belirleyen mesaj
)
from langchain.chat_models import ChatOpenAI


try:
  openai_apikey = userdata.get('OPENAI_APIKEY')
  print('OpenAI anahtarın buldum hacım..')
except Exception as e :
  print('Anahtarı bulamadım hacı.')
  print(str(e))

OpenAI anahtarın buldum hacım..


In [ ]:
chat = ChatOpenAI(
    api_key=openai_apikey,
    model='gpt-3.5-turbo',
    temperature=0.5,
    max_tokens=1024,
    max_retries=3
)

messages = [
    SystemMessage(content='Sen Fizik alanında nobel ödülü almış bir bilim adamısın'),
    HumanMessage(content='Kuantum dolanıklığını tek bir cümle ile açıkla'),
]
output = chat(messages = messages)
print(output)
%pycat output.content

In [ ]:
messages_second = [
    SystemMessage(content="""
    Sen çok iyi bir astrofizikçi, uzay bilimci ve akademisyensin. Sana sorulan kendi alanında (astrofizik ve uzay bilimi) soruları cevaplayacak,
          bilmediğin ve alanına girmeyen soruları cevaplamayacaksın (kibarca sebebini söyle).
    """),
    HumanMessage(content='Bana kurufasulyenin en iyi tarifini verir misin?')
]
output = chat(messages_second)
%pycat output.content

#### JSON çıktısı alıp işlem yapma

In [ ]:
system_msg = """
Sen dünyanın en geniş ve güçlü internet arama makinesisin ve kesinlikle yalancı değilsin. Sana sorulan konularda örnek siteleri aşağıdaki json formatına uygun olarak json formatında cevaplayacaksın. "alaka_duzeyi" alanı sonucunu getirdiğin sitenin yapılan sorguyla ile alakasının 10 ile 1 arasında değerlendirmesidir. 1 en alakasız, 10 en alakalı şeklinde, bu değer tekrar edebilir. Sorgular yarım olsa bile en yakın anlamdaki siteleri getir. "kısa_aciklama" alanı 216 karakteri geçmeyecek şekilde, ilgili internet sitesinin açıklaması olacak.

{
    "sonuc_sayisi": "bulunan sonuc sayılarını buraya yaz",
    "yorum": "buraya kendi kısa yorumunu yazacaksın",
    "sonuclar": [
        {
            "alaka_duzeyi": 10,
            "diller": ["Türkçe", "Çok dilli"],
            "kisa_aciklama": "Bu sitede internet araması yapılabilir",
            "adres": "https://www.google.com"
        },
        {
            "alaka_duzeyi": 10,
            "diller": ["İngilizce"],
            "kisa_aciklama": "Bu sitede fizik problemlerinin formülasyonunu ve çözümlerini bulabilirsiniz.",
            "adres": "https://www.physicsforums.com"
        },
        {
            "alaka_duzeyi": 5,
            "diller": ["Türkçe", "Çok dilli"],
            "kisa_aciklama": "bu sitede video izlenebilir",
            "adres": "https://www.youtube.com"
        },
    ],
}
----
----
bilmediğin konularda aşağıdaki gibi bir json sonucu vereceksin:

{
"sonuc_sayisi": 0,
"Sebep": "Aranan kriterde site bulunamadı. Bunu daha iyi anlatabilirsin"
}
"""

messages = [
    SystemMessage(content= system_msg ),
    HumanMessage(content='Alkolsüz kokteyl tarifleri alabileceğim internet siteleri?')
]
output = chat(messages = messages)

In [ ]:
%pycat output.content

In [ ]:
type(output.content)

str

In [ ]:
import json

In [ ]:
json_data = json.loads(output.content)
print(type(json_data))
print(json_data.keys())

<class 'dict'>
dict_keys(['sonuc_sayisi', 'yorum', 'sonuclar'])


In [ ]:
sonuc_sayisi = int(json_data.get('sonuc_sayisi'))
sonuc_sayisi

3

In [ ]:
yorum = json_data.get('yorum')
yorum

'Alkolsüz kokteyl tarifleri için aşağıdaki siteleri inceleyebilirsiniz.'

In [ ]:
sonuclar: list = json_data.get('sonuclar')
len(sonuclar)

3

In [ ]:
sonuc_sayisi = int(json_data.get('sonuc_sayisi', 0))
if sonuc_sayisi == 0:
  print('Sonuç bulkunamadı...')
else:
  yorum = json_data.get('yorum')
  print(yorum)
  sonuclar = json_data.get('sonuclar')
  if isinstance(sonuclar, list):
    if len(sonuclar) > 0:
      for e, s in enumerate(sonuclar, start=1):
        print("{} - {}".format(e, s.get('adres')))
        print('\t\t {}'.format(s.get('kisa_aciklama')))
        print('\t\t Alaka düzeyi: {}'.format(s.get('alaka_duzeyi')))
        print('\t\t {}'.format(
          '-'.join([d for d in s.get('diller')])
        ))

Alkolsüz kokteyl tarifleri için aşağıdaki siteleri inceleyebilirsiniz.
1 - https://www.kokteylrehberi.com/alkolsuz-kokteyl-tarifleri
		 Bu sitede çeşitli alkolsüz kokteyl tarifleri bulabilirsiniz.
		 Alaka düzeyi: 10
		 Türkçe
2 - https://www.alkolsuzkokteyller.com
		 Bu sitede alkolsüz kokteyl tarifleri ve içecek önerileri bulabilirsiniz.
		 Alaka düzeyi: 8
		 Türkçe
3 - https://www.alkolsuzkokteyl.com
		 Bu sitede sağlıklı ve lezzetli alkolsüz içecek tarifleri bulabilirsiniz.
		 Alaka düzeyi: 7
		 Türkçe


### Fonksiyona Çevirelim

In [ ]:

def start_search(chat, user_message:str) -> dict:
  system_msg = """
        Sen dünyanın en geniş ve güçlü internet arama makinesisin ve kesinlikle yalancı değilsin. Sana sorulan konularda örnek siteleri aşağıdaki json formatına uygun olarak json formatında cevaplayacaksın. "alaka_duzeyi" alanı sonucunu getirdiğin sitenin yapılan sorguyla ile alakasının 10 ile 1 arasında değerlendirmesidir. 1 en alakasız, 10 en alakalı şeklinde, bu değer tekrar edebilir. Sorgular yarım olsa bile en yakın anlamdaki siteleri getir. "kısa_aciklama" alanı 216 karakteri geçmeyecek şekilde, ilgili internet sitesinin açıklaması olacak.

        {
            "sonuc_sayisi": "bulunan sonuc sayılarını buraya yaz",
            "yorum": "buraya kendi kısa yorumunu yazacaksın",
            "sonuclar": [
                {
                    "alaka_duzeyi": 10,
                    "diller": ["Türkçe", "Çok dilli"],
                    "kisa_aciklama": "Bu sitede internet araması yapılabilir",
                    "adres": "https://www.google.com"
                },
                {
                    "alaka_duzeyi": 10,
                    "diller": ["İngilizce"],
                    "kisa_aciklama": "Bu sitede fizik problemlerinin formülasyonunu ve çözümlerini bulabilirsiniz.",
                    "adres": "https://www.physicsforums.com"
                },
                {
                    "alaka_duzeyi": 5,
                    "diller": ["Türkçe", "Çok dilli"],
                    "kisa_aciklama": "bu sitede video izlenebilir",
                    "adres": "https://www.youtube.com"
                },
            ],
        }
        ----
        ----
        bilmediğin konularda aşağıdaki gibi bir json sonucu vereceksin:

        {
        "sonuc_sayisi": 0,
        "Sebep": "Aranan kriterde site bulunamadı. Bunu daha iyi anlatabilirsin"
        }
        """

  messages = [
        SystemMessage(content= system_msg ),
        HumanMessage(content=user_message)
    ]
  output = chat(messages = messages)
  return json.loads(output.content)


def run(user_message:str) -> None:
  chat = ChatOpenAI(
      api_key=openai_apikey,
      model='gpt-4', # gpt-3.5-turbo
      temperature=0.5,
      max_tokens=1024,
      max_retries=3
      )
  json_data = start_search(chat, user_message)
  sonuc_sayisi = int(json_data.get('sonuc_sayisi', 0))
  if sonuc_sayisi == 0:
    print('Sonuç bulkunamadı...')
  else:
    yorum = json_data.get('yorum')
    print(yorum)
    sonuclar = json_data.get('sonuclar')
    if isinstance(sonuclar, list):
      if len(sonuclar) > 0:
        for e, s in enumerate(sonuclar, start=1):
          print("{} - {}".format(e, s.get('adres')))
          print('\t\t {}'.format(s.get('kisa_aciklama')))
          print('\t\t Alaka düzeyi: {}'.format(s.get('alaka_duzeyi')))
          print('\t\t {}'.format(
            '-'.join([d for d in s.get('diller')])
          ))



In [ ]:
user_message = 'Kuantum fiziğini öğrenebileceğim internet sayfları'
run(user_message = user_message)

Kuantum fiziği üzerine eğitimler ve bilgiler sunan siteleri listeledim.
1 - https://quantummechanics.stanford.edu
		 Stanford Üniversitesi'nin kuantum mekaniği dersleri ve kaynaklarına erişim sağlar.
		 Alaka düzeyi: 10
		 İngilizce
2 - https://www.quantum-physics.polytechnique.fr
		 Kuantum fiziği üzerine çeşitli dersler ve bilgiler içerir.
		 Alaka düzeyi: 9
		 İngilizce
3 - https://www.fiziktamamen.com
		 Kuantum fiziği üzerine Türkçe ve İngilizce dersler ve bilgiler sunar.
		 Alaka düzeyi: 8
		 Türkçe-Çok dilli
